In [26]:
import spacy
import pandas as pd
from spacy.matcher import PhraseMatcher


In [17]:
nlp = spacy.blank("es")

In [28]:
df_gazetteer = pd.read_csv('data/distemist_gazeteer_umls_spanish.tsv', sep='\t')
df_gazetteer.head()

,code,term
0,9989000,anomalía congénita de dedo del pie
1,9989000,malformación congénita de dedo del pie
2,9984005,exfoliación de dientes por enfermedad sistémica
3,9982009,intoxicación causada por cocaína
4,998008,enfermedad de Chagas con compromiso del corazón


In [33]:
gazetteer_list = list(df_gazetteer['term'].values)
gazetteer_codes = list(df_gazetteer['code'].values)

In [41]:
len(gazetteer_list)

285930

In [39]:
matcher = PhraseMatcher(nlp.vocab)
label = 'ENFERMEDAD'

for code, text, doc in zip(gazetteer_codes, gazetteer_list, nlp.tokenizer.pipe(gazetteer_list)):
    matcher.add(f'{code}_{text}', None, doc)

In [40]:
len(matcher)

285930

In [45]:
text = "Los antecedentes personales de este paciente son: Independiente para actividades básicas de la vida diaria (ABVD). Fumador. Hipertensión arterial (HTA), dislipemia, diabetes mellitus tipo 2 en tratamiento con antidiabéticos orales (ADO). Enfermedad renal crónica (ERC) estadio 3A. Enfermedad pulmonar obstructiva crónica (EPOC) grave."

In [46]:
doc = nlp(text)
matches = matcher(doc, as_spans=True)

In [54]:
from spacy.util import filter_spans

filtered_matches = filter_spans(matches)

In [55]:
for match in filtered_matches:
    print(doc[match.start:match.end], doc[match.start].idx, doc[match.end-1].idx + len(doc[match.end-1]))

Fumador 115 122
Hipertensión arterial 124 145
diabetes mellitus tipo 2 165 189
Enfermedad renal crónica 238 262
Enfermedad pulmonar obstructiva crónica 281 320
EPOC 322 326


In [56]:
import pickle

with open('data/diseases.pickle', 'wb') as handle:
    pickle.dump(matcher, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [50]:
# test load
with open('data/diseases.pickle', 'rb') as handle:
    matcher2 = pickle.load(handle)

In [68]:
nlp2 = spacy.blank("es", vocab=matcher2.vocab) # vocab from reloaded PhraseMatcher

print(matcher2(nlp("Explica dolor abdominal de 2 setmanes d’evolució i basques sense vòmits, es programa visita presencial per a l’endemà.")))

[(5339602756129875399, 1, 3)]
